In [1]:
# Cell 1
import sys
import os


# This ensures imports like `from utils.data_loader import load_data` work
project_root = os.path.abspath("..")  # Adjust if your notebook is deeper
if project_root not in sys.path:
 sys.path.insert(0, project_root)


import pandas as pd
import numpy as np
import json
from datetime import datetime
import git

from utilis.data_loader import load_data
from strategies.current_algo import AdvancedTradingAlgorithm
from backtests.engine import run_backtest
from backtests.metrics import calculate_comprehensive_metrics


In [5]:
# Cell 2
def establish_baseline():
    symbols = ['AAPL', 'MSFT', 'SPY', 'GOOGL']
    periods = ['1y', '2y']
    baseline_results = {}

    for symbol in symbols:
        for period in periods:
            print(f"Running baseline: {symbol} - {period}")
            
            data = load_data(symbol, period)
            algo = AdvancedTradingAlgorithm()
            results = run_backtest(algo, data)
            metrics = calculate_comprehensive_metrics(results)

            key = f"{symbol}_{period}"
            baseline_results[key] = {
                'symbol': symbol,
                'period': period,
                'total_return': metrics['total_return'],
                'sharpe_ratio': metrics['sharpe_ratio'],
                'max_drawdown': metrics['max_drawdown'],
                'win_rate': metrics['win_rate'],
                'profit_factor': metrics['profit_factor'],
                'total_trades': metrics['total_trades'],
                'timestamp': datetime.now().isoformat(),
            
            }

    with open('../backtests/results/baseline/baseline_results.json', 'w') as f:
        json.dump(baseline_results, f, indent=2)

    return baseline_results

baseline = establish_baseline()


Running baseline: AAPL - 1y
Looking for file at: /workspaces/anomaly_trading_algorithm/data/processed/AAPL_1y.csv
Running baseline: AAPL - 2y
Looking for file at: /workspaces/anomaly_trading_algorithm/data/processed/AAPL_2y.csv
Running baseline: MSFT - 1y
Looking for file at: /workspaces/anomaly_trading_algorithm/data/processed/MSFT_1y.csv
Running baseline: MSFT - 2y
Looking for file at: /workspaces/anomaly_trading_algorithm/data/processed/MSFT_2y.csv
Running baseline: SPY - 1y
Looking for file at: /workspaces/anomaly_trading_algorithm/data/processed/SPY_1y.csv
Running baseline: SPY - 2y
Looking for file at: /workspaces/anomaly_trading_algorithm/data/processed/SPY_2y.csv


Running baseline: GOOGL - 1y
Looking for file at: /workspaces/anomaly_trading_algorithm/data/processed/GOOGL_1y.csv
Running baseline: GOOGL - 2y
Looking for file at: /workspaces/anomaly_trading_algorithm/data/processed/GOOGL_2y.csv
